In [1]:
import pandas as pd
import numpy as np

### Data Simulation
- Probability sampling method = We don’t have actual N people’s data, so we generate it based on real-world probability

In [2]:
np.random.seed(1140) #for reproducibility
N = 1000

### Age range = 50 to 60 
Why?
- Typical retirement age for majority of Malaysian (targeting the group with real urgency)
- Following the standard of almost all public datasets (EPF, NHMS, DOSM) 
- Actionable scoring logic (immediate usage)

In [3]:
age = np.random.randint(50, 61, size=N)

### Gender 
Males: est 17.5 million (49%) ,Females: est 15.9 million (51%)

Source = https://www.dosm.gov.my/site/downloadrelease?admin_view=&id=current-population-estimates-malaysia----2023&lang=English&

In [4]:
gender = np.random.choice(["Male", "Female"], size=N, p=[0.49, 0.51])

### State
Derived population proportions for each Malaysian state based on DOSM 2024

Source = https://open.dosm.gov.my/data-catalogue/population_state

In [5]:
URL_DATA = 'https://storage.dosm.gov.my/population/population_state.parquet'
data = pd.read_parquet(URL_DATA)
df_2024 = data[
    (pd.to_datetime(data['date']).dt.year == 2024) &
    (data['sex'] == 'both') &
    (data['age'] == 'overall') &
    (data['ethnicity'] == 'overall')
]

# compute proportion
total_pop_2024 = df_2024.groupby('state')['population'].sum().reset_index()
total_pop_2024 = total_pop_2024.sort_values(by='population', ascending=False)
total = total_pop_2024['population'].sum()
total_pop_2024['population_pct'] = total_pop_2024['population'] / total 
#there are already in percentage but does not sum up to 1 due to floating-point issues. (sum = 0.999)

states = total_pop_2024['state'].tolist()
weights = total_pop_2024['population_pct'].to_numpy()  #convert to numpy array to support element wise vector operation 
weights = weights / weights.sum()  #  normalize again to fix small rounding (sum up to 1)

states = np.random.choice(states, size=N, p=weights)

### Household Size
Household size based on typical Malaysian distribution. 
1. Average is 3.8 ranging from 1 to 6
2. As for probability, assign discrete probability mass function (PMF) based on grouped statistical aggregates

To understand number 2:
Based on DOSM approximate breakdown:

    • 1–2 persons: ~20%
    
    • 3–4 persons: ~50%
    
    • > 5 persons: ~30%
    
To Simulate our data into imagine a histogram that match DOSM breakdown:

-Divide the household sizes into bins like this:[1,2,3,4,5,6] 

-Assign probabilities to each bin that reflect the target ranges (must adding up to 100%): [10%,15%,25%,25%,15%,10%]

This matche the expected shape of household sizes in Malaysia while still keep the realistic average (3.8), the shape and proper probability rule (sum to 1)

Source :

Population and Housing Census : https://www.dosm.gov.my/site/downloadrelease?admin_view=&id=key-findings-population-and-housing-census-of-malaysia-2020-administrative-district&lang=English

DOSM : https://open.dosm.gov.my/dashboard/kawasanku

In [6]:
household_size = np.random.choice([1, 2, 3, 4, 5, 6], size=N, p=[0.1, 0.15, 0.25, 0.25, 0.15, 0.1])

### Monthly Income

Statistical method : Stratified Sampling 

In simple words, split them into meaningful group an then sample from each group

Step:
1. Group into 10 income brackets using real 2022 DOSM data (dosm tabulate by percentile aka 100 equally sized group so use decile)
2. Randomly picked one bracket for each simulated person so each income group gets fair representation
3. Randomly generated an income inside that group’s actual range (for natural variation)
   
source = https://open.dosm.gov.my/data-catalogue/hh_income

In [8]:
URL = 'https://storage.dosm.gov.my/hies/hies_malaysia_percentile.parquet'
df = pd.read_parquet(URL)
df = df[(df['date']=='2022-01-01') & (df['variable']=='median')]

# Derive decile cutoffs
df = df.sort_values('percentile') #sort in order
cut = {0: int(df['income'].min())} # start from lowest income
for p in range(10, 101, 10): # income cutoff for every decile 
    cut[p] = int(df[df['percentile'] == p]['income'].iloc[0])
ranges = [(cut[i], cut[i+10]) for i in range(0,100,10)] #each tuple is the income range for 1 decile
probs = [0.1] * 10 #10% chance for each decile assume equal population share

# Simulate Income Based on These Decile Ranges
def simulate_income_empirical():
    d = np.random.choice(10, p=probs)
    return np.random.randint(ranges[d][0], ranges[d][1] + 1) #gives variability inside each income group to make it more human

monthly_income = np.array([simulate_income_empirical() for _ in range(N)])

### EPF Balance
Based on Official KWSP Annual report median EPF balances of RM 35,161 (age 50–54) and RM 11,547 (age 55–59) however these reflect the entire Malaysian population including informal workers and early withdrawals. For this simulation use: 

Age 50–54: RM50k

Age 55–59: RM70k

Why? This better represents the audience likely to use this tool (stable, full-time workers (M40/T20)). We believe this assumption improves realism to
plan for retirement.

Next, most people are near average while some are higher or lower so we use normal distribution

Estimate the standard deviation using coefficient of variation (how spread data is relative to the average). Take 0.20 and 0.18. In economic modeling, a typical assumption is that income or savings values for middle-income groups have a standard deviation around 15%–25% of the mean. This keeps the simulated EPF balances realistic without overfitting or artificial precision.

Source : https://www.kwsp.gov.my/documents/d/guest/2023-epf-dividend_appendix_2

In [9]:
def simulate_epf(age):
    if age < 55:
        return int(np.random.normal(50000, 9000))
    else:
        return int(np.random.normal(70000, 14000))

epf_balance = np.array([simulate_epf(a) for a in age])

### Debt amount as % of income (varies 20% to 80%)
Based on BNM debt servicing ratio 

Source = https://www.bnm.gov.my/publications/fsr2023h2

In [10]:
debt_amount = np.array([int(income * np.random.uniform(0.2, 0.8)) for income in monthly_income])

### Chronic Disease Probability Simulation

Method : Probabilistic Sampling with Binomial Distribution

Based on NHMS 2023 key findings :

1. Diabetes prevalence ~15.6%
2. Hypertension prevalence ~29.2%

Therefore we use baseline 35% (Among Malaysians aged 50–59, combined chronic disease risk possibility (at least one))

To reflect age-related risk:
- Let the probability increase by 1% per year over age 50 
- Males receive a +3% adjustment based on higher male risk in NHMS
- Probabilities were capped at 85% to avoid unrealistic outcomes (no population has that high a chronic rate, at least in Malaysia)

Source = https://iku.nih.gov.my/images/nhms2023/key-findings-nhms-2023.pdf

In [11]:
def simulate_chronic(age, gender):
    base_prob = 0.35 + 0.01 * (age - 50)
    if gender == "Male":
        base_prob += 0.03
    return np.random.binomial(1, min(base_prob, 0.85))

has_chronic_disease = np.array([simulate_chronic(a, g) for a, g in zip(age, gender)])

### Medical expense monthly (depends on chronic)
Based on NHMS 2023: average outpatient OOP = RM134 but this reflects a central tendency not representative of people with chronic diseases. So we take it as baseline.

1. Medication + visits + tests = RM200–800/month is realistic for chronic conditions
   
   Lower bound (RM200): Represents public hospital patients with subsidies.
   
   Upper bound (RM800): Covers patients using private clinics with multiple medications and those managing multiple conditions.
   
2. Healthy individuals assigned RM50–200 to simulate light outpatient or supplements

Source = https://iku.nih.gov.my/images/nhms2023/fact-sheet-nhms-2023.pdf

In [12]:
medical_expense_monthly = np.where(
    has_chronic_disease == 1,
    np.random.randint(200, 800, size=N),
    np.random.randint(50, 200, size=N)
)
# use uniform distribution to assume every number in that range equally likely to occur

### Mental stress level (0 to 1) → higher if low income or sick
Based on NHMS 2015 shows nearly 30% of Malaysians had mental health issues, especially among the economically vulnerable. WHO findings also indicate there are significant rising trend of mental health issue among Malaysian since recent years

Model Logic:

1. Low income (< RM3,000) = Financial stress (+0.2)
   
Based on global and Malaysian findings: Low-income individuals consistently show higher rates of depression and anxiety

2. Chronic disease = higher stress (+0.1)

Chronic disease leads to living with illness plus financial burden which may drive psychological burden (seen in NHMS and mental health research)

3. Random base stress (0.3–0.7) accounts for typical variance in mental wellbeing even without financial or health pressures
   
Source:
- https://www.who.int/about/accountability/results/who-results-report-2024-2025/country-profile/2024/malaysia
- https://pmc.ncbi.nlm.nih.gov/articles/PMC8863240/
- https://www.moh.gov.my/moh/resources/Penerbitan/Rujukan/NCD/National%20Strategic%20Plan/The_National_Strategic_Plan_For_Mental_Health_2020-2025.pdf
- https://e-journal.uum.edu.my/index.php/jps/article/view/14522

In [13]:
mental_stress_level = []
for income, chronic in zip(monthly_income, has_chronic_disease):
    stress = np.random.uniform(0.3, 0.7)
    if income < 3000:
        stress += 0.2
    if chronic:
        stress += 0.1
    mental_stress_level.append(round(min(stress, 1.0), 2))


###  Expected monthly expenses based on household size with CPI state multiplier

CPI = Consumer Price Index = Price change over time or simply put a metric for inflation

How DOSM get this? Every month they compares the current total price of items in each state to what it cost in a base year (2010)

We normalize CPI to get multiplier (national average CPI as reference)

- more expensive than average therefor multiplier > 1
- cheaper then multiplier < 1

Base expense :

- Let take 500 to 800 for basic (rent, electric,water,food)
- Then take add 300 per household (variable cost for each extra person)
  
[DOSM 2022 = house of 4 spend around 5k+ so that is 1k+ each so base = (within 500–800) + (4 × 300) = RM 1k+ #allign with real figure]

- Applies multiplier to adjust for cost of living per state (by default = 1)
  
Source = https://open.dosm.gov.my/data-catalogue/cpi_state

In [17]:
df_cpi = pd.read_parquet("https://storage.dosm.gov.my/cpi/cpi_2d_state.parquet")
latest = df_cpi[df_cpi['division'] == 'overall'].sort_values('date').drop_duplicates('state', keep='last')#CPI has many categories so filter only overall
latest = latest[['state', 'index']] # base year 2010 = 100
ref = latest['index'].mean() # take the national average as reference 
latest['multiplier'] = latest['index'] / ref #Normalize
cpi_map = latest.set_index('state')['multiplier'].to_dict()
states = list(cpi_map.keys())
household_size = np.random.choice([1, 2, 3, 4, 5, 6], size=N, p=[0.1, 0.15, 0.25, 0.25, 0.15, 0.1]) #from household simulation 
state = np.random.choice(states, size=N, p=[1/len(states)]*len(states))  # randomly assigned for now

#Simulate base expenses with CPI adjustment
base_expense = []
for h, s in zip(household_size, state):
    cpi = cpi_map.get(s, 1.0)  # default to 1.0 if missing
    base = np.random.randint(500, 800) + h * 300
    base_expense.append(int(base * cpi))


### Latent retirement readiness score (probabilistic)
The score is computed using a logistic formula 

Why?
- Turns any real number into between 0 to 1 which can then just multiply with 100 to get percentage
- Smooth function so small changes in variable cause small change in result (no suprising outcome)
- Easy to understand weight and make decision (feasy to seelike debt can hurt more than household (example only) )

This is a common way actually in probabilities, you can explore more on google or just ask AI

In [15]:
readiness_score = []
for i in range(N):
    logit = (
        + 0.00005 * epf_balance[i]
        - 0.0002 * debt_amount[i]
        - 0.5 * has_chronic_disease[i]
        - 0.3 * mental_stress_level[i]
        + 0.2 * np.log1p(monthly_income[i])
    )
    p = 1 / (1 + np.exp(-logit))
    readiness_score.append(int(round(p * 100)))

### Export to csv

In [16]:
df = pd.DataFrame({
    "age": age,
    "gender": gender,
    "state": state,
    "monthly_income": monthly_income,
    "epf_balance": epf_balance,
    "debt_amount": debt_amount,
    "household_size": household_size,
    "has_chronic_disease": has_chronic_disease,
    "medical_expense_monthly": medical_expense_monthly,
    "mental_stress_level": mental_stress_level,
    "expected_monthly_expense": base_expense,
    "retirement_readiness_score": readiness_score
})

df.to_csv("retirement.csv", index=False)
print("retirement.csv saved")


retirement.csv saved
